# ***`ETL Pipeline for Analyzing Market Capitalization of the World's Largest Banks`***







# ***Importing All Necessary Libraries***

In [70]:
import pandas as pd
import numpy as np
import sqlite3
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# ***Function to Log Process***

In [71]:
def log_progress(message):
    """Logs the progress of the code to a text file in the specified format."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"{timestamp} : {message}\n"

    # Write the log entry to the file
    with open("code_log.txt", "a") as log_file:
        log_file.write(log_entry)

log_progress("Preliminaries complete. Initiating ETL process")


# ***Extract***

In [72]:
def extract(url):
    """Extracts the table under 'By market capitalization' and returns a Pandas DataFrame."""
    log_progress("Fetching webpage...")
    response = requests.get(url)
    if response.status_code != 200:
        log_progress(f"Failed to fetch data from {url}")
        raise Exception("Failed to fetch data from the website")

    log_progress("Webpage fetched successfully. Parsing HTML...")
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the table
    table = soup.find('table', {'class': 'wikitable sortable'})
    if not table:
        log_progress("Failed to find the table with the class 'wikitable sortable'.")
        raise Exception("Table not found in the webpage")

    log_progress("Table found. Reading data into a DataFrame...")
    df = pd.read_html(str(table))[0]

    df.rename(columns={'Name': 'Name', 'Market cap(US$ billion)': 'MC_USD_Billion'}, inplace=True)

    df['MC_USD_Billion'] = df['MC_USD_Billion'].astype(str).str.replace(r'\n', '', regex=True).astype(float)

    log_progress("Data extraction complete. Returning DataFrame.")
    return df


In [73]:

!cat code_log.txt

from google.colab import files
files.download('code_log.txt')


2024-12-26 23:26:35 : Data extraction complete. Initiating Transformation process
2024-12-26 23:26:46 : Data transformation complete. Returning the transformed DataFrame
2024-12-26 23:29:40 : Starting data extraction process.
2024-12-26 23:29:41 : Table not found on the webpage.
2024-12-26 23:33:11 : Starting data extraction process.
2024-12-26 23:33:12 : Table not found on the webpage.
2024-12-26 23:33:52 : Starting data extraction process.
2024-12-26 23:33:53 : Table not found on the webpage.
2024-12-26 23:36:33 : Starting data extraction process.
2024-12-26 23:38:07 : Starting data extraction process.
2024-12-26 23:38:08 : Number of tables found: 3
2024-12-26 23:38:08 : Columns after renaming: Index(['Rank', 'Bank name', 'MC_USD_Billion'], dtype='object')
2024-12-26 23:38:08 : Data extraction complete. Initiating transformation process.
2024-12-26 23:38:17 : Starting data extraction process.
2024-12-26 23:38:17 : Number of tables found: 3
2024-12-26 23:38:17 : Columns after renaming

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [74]:
def extract(url):
    """Extracts the table under 'By market capitalization' and returns a Pandas DataFrame."""
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup

    # Fetch the webpage
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception("Failed to fetch the webpage")

    # Read all tables
    tables = pd.read_html(response.text)
    print(f"Number of tables found: {len(tables)}")

    # Assuming the table of interest is at index 0
    df = tables[0]

    # Debug: Print column names
    print("Column names in the DataFrame:", df.columns)

    return df


In [75]:
def extract(url):
    """Extracts the table under 'By market capitalization' and returns a Pandas DataFrame."""
    import pandas as pd
    import requests

    # Fetch the webpage
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception("Failed to fetch the webpage")

    # Read all tables
    tables = pd.read_html(response.text)

    # Assuming the table of interest is at index 0
    df = tables[0]

    # Debug: Print the raw DataFrame
    print("Raw DataFrame:\n", df.head())

    # Dynamically rename columns
    for col in df.columns:
        if 'Name' in col:
            df.rename(columns={col: 'Name'}, inplace=True)
        if 'Market cap' in col:
            df.rename(columns={col: 'MC_USD_Billion'}, inplace=True)

    # Debug: Print updated DataFrame
    print("Updated DataFrame:\n", df.head())

    # Convert the market cap column to float (if applicable)
    if 'MC_USD_Billion' in df.columns:
        df['MC_USD_Billion'] = df['MC_USD_Billion'].astype(str).str.replace(r'\n', '', regex=True).astype(float)

    return df


In [76]:
def extract(url):
    """Extracts the table under 'By market capitalization' and returns a Pandas DataFrame."""
    import pandas as pd
    import requests

    # Fetch the webpage
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception("Failed to fetch the webpage")

    # Read all tables
    tables = pd.read_html(response.text)

    # Assuming the table of interest is at index 0
    df = tables[0]

    # Dynamically rename columns using case-insensitive matching
    df.rename(columns=lambda col: 'Name' if 'name' in col.lower() else col, inplace=True)
    df.rename(columns=lambda col: 'MC_USD_Billion' if 'market cap' in col.lower() else col, inplace=True)

    # Debug: Print the updated DataFrame
    print("Updated DataFrame:\n", df.head())

    # Convert the market cap column to float (if applicable)
    if 'MC_USD_Billion' in df.columns:
        df['MC_USD_Billion'] = df['MC_USD_Billion'].astype(str).str.replace(r'\n', '', regex=True).astype(float)

    return df


In [77]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
df = extract(url)
print(df.head())


Updated DataFrame:
    Rank                                     Name  MC_USD_Billion
0     1                           JPMorgan Chase          432.92
1     2                          Bank of America          231.52
2     3  Industrial and Commercial Bank of China          194.56
3     4               Agricultural Bank of China          160.68
4     5                                HDFC Bank          157.91
   Rank                                     Name  MC_USD_Billion
0     1                           JPMorgan Chase          432.92
1     2                          Bank of America          231.52
2     3  Industrial and Commercial Bank of China          194.56
3     4               Agricultural Bank of China          160.68
4     5                                HDFC Bank          157.91


<ipython-input-76-4665b41071d2>:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


# ***Transform***

In [78]:
def log_progress(message):
    """Logs the progress of the code to a text file and displays it inline."""
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"{timestamp} : {message}\n"

    # Save logs to a file
    with open("code_log.txt", "a") as log_file:
        log_file.write(log_entry)

    # Display the log in the Colab output
    print(log_entry)

def transform(df, csv_path):
    """
    Transforms the DataFrame by adding columns for GBP, EUR, and INR market capitalizations.
    """
    # Step 1: Read the exchange rate CSV into a dictionary
    exchange_rate_data = pd.read_csv(csv_path)
    exchange_rate = dict(zip(exchange_rate_data['Currency'], exchange_rate_data['Rate']))

    # Step 2: Add transformed columns
    df['MC_GBP_Billion'] = [np.round(x * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate['INR'], 2) for x in df['MC_USD_Billion']]

    # Log the transformation process
    log_progress("Data transformation complete. Returning the transformed DataFrame.")
    return df

url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
exchange_csv_path = "exchange_rate.csv"


df = extract(url)

df_transformed = transform(df, exchange_csv_path)


print(df_transformed)


print("Market capitalization of the 5th largest bank in billion EUR:", df_transformed['MC_EUR_Billion'][4])


Updated DataFrame:
    Rank                                     Name  MC_USD_Billion
0     1                           JPMorgan Chase          432.92
1     2                          Bank of America          231.52
2     3  Industrial and Commercial Bank of China          194.56
3     4               Agricultural Bank of China          160.68
4     5                                HDFC Bank          157.91
2024-12-27 03:10:24 : Data transformation complete. Returning the transformed DataFrame.

   Rank                                     Name  MC_USD_Billion  \
0     1                           JPMorgan Chase          432.92   
1     2                          Bank of America          231.52   
2     3  Industrial and Commercial Bank of China          194.56   
3     4               Agricultural Bank of China          160.68   
4     5                                HDFC Bank          157.91   
5     6                              Wells Fargo          155.87   
6     7                 

<ipython-input-76-4665b41071d2>:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


# ***Load***

In [79]:
def load_to_csv(df, output_path):
    """
    Saves the transformed DataFrame to a CSV file at the specified path.
    """
    try:
        # Save the DataFrame to the CSV file
        df.to_csv(output_path, index=False)

        # Log the success message
        log_progress(f"Data saved to CSV file at {output_path}")
        print(f"Data successfully saved to {output_path}")
    except Exception as e:
        log_progress(f"Failed to save data to CSV: {e}")
        raise

# Example usage
output_csv_path = "Largest_banks_data.csv"  # Path for saving the CSV file

# Assuming the transformed DataFrame `df_transformed` is already available
load_to_csv(df_transformed, output_csv_path)


2024-12-27 03:10:29 : Data saved to CSV file at Largest_banks_data.csv

Data successfully saved to Largest_banks_data.csv


In [80]:
def load_to_db(df, db_name, table_name):
    """
    Loads the transformed DataFrame into an SQLite database table.
    """
    try:
        # Establish the connection to the SQLite database
        conn = sqlite3.connect(db_name)
        log_progress(f"SQL Connection initiated to {db_name}")

        # Save the DataFrame to the database
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        log_progress(f"Data loaded to database '{db_name}' in table '{table_name}'")

        # Close the connection
        conn.close()
        log_progress(f"SQL Connection to {db_name} closed successfully")
        print(f"Data successfully loaded into the '{table_name}' table in database '{db_name}'")

    except Exception as e:
        log_progress(f"Failed to load data into the database: {e}")
        raise

# Example usage
db_name = "Banks.db"  # Database name
table_name = "Largest_banks"  # Table name

# Assuming the transformed DataFrame `df_transformed` is already available
load_to_db(df_transformed, db_name, table_name)


2024-12-27 03:10:32 : SQL Connection initiated to Banks.db

2024-12-27 03:10:32 : Data loaded to database 'Banks.db' in table 'Largest_banks'

2024-12-27 03:10:32 : SQL Connection to Banks.db closed successfully

Data successfully loaded into the 'Largest_banks' table in database 'Banks.db'


In [81]:
def load_to_csv(df, output_path):
    """
    Saves the transformed DataFrame to a CSV file at the specified path.
    """
    try:
        df.to_csv(output_path, index=False)
        log_progress(f"Data saved to CSV file at {output_path}")
        print(f"Data successfully saved to {output_path}")
    except Exception as e:
        log_progress(f"Failed to save data to CSV: {e}")
        raise

def load_to_db(df, db_name, table_name):
    """
    Loads the transformed DataFrame into an SQLite database table.
    """
    try:
        conn = sqlite3.connect(db_name)
        log_progress(f"SQL Connection initiated to {db_name}")
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        log_progress(f"Data loaded to database '{db_name}' in table '{table_name}'")
        conn.close()
        log_progress(f"SQL Connection to {db_name} closed successfully")
        print(f"Data successfully loaded into the '{table_name}' table in database '{db_name}'")
    except Exception as e:
        log_progress(f"Failed to load data into the database: {e}")
        raise


# ***Function to execute queries on the database***

In [82]:
def run_queries(query_statement, db_name):
    """
    Executes the given SQL query on the specified database and prints the results.
    """
    try:
        # Establish the database connection
        conn = sqlite3.connect(db_name)
        log_progress(f"Executing query: {query_statement}")

        # Execute the query and fetch results
        query_result = pd.read_sql_query(query_statement, conn)
        print(f"Query:\n{query_statement}\n")
        print("Output:")
        print(query_result)

        # Log the completion of query execution
        log_progress("Query executed successfully.")

        # Close the connection
        conn.close()
        log_progress(f"SQL Connection to {db_name} closed after query execution.")

    except Exception as e:
        log_progress(f"Error while executing query: {e}")
        raise


db_name = "Banks.db"

# Query 1: Print the contents of the entire table
query_1 = "SELECT * FROM Largest_banks"
run_queries(query_1, db_name)

# Query 2: Print the average market capitalization in Billion GBP
query_2 = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_queries(query_2, db_name)

# Query 3: Print only the names of the top 5 banks
query_3 = "SELECT Name FROM Largest_banks LIMIT 5"
run_queries(query_3, db_name)


2024-12-27 03:10:39 : Executing query: SELECT * FROM Largest_banks

Query:
SELECT * FROM Largest_banks

Output:
   Rank                                     Name  MC_USD_Billion  \
0     1                           JPMorgan Chase          432.92   
1     2                          Bank of America          231.52   
2     3  Industrial and Commercial Bank of China          194.56   
3     4               Agricultural Bank of China          160.68   
4     5                                HDFC Bank          157.91   
5     6                              Wells Fargo          155.87   
6     7                        HSBC Holdings PLC          148.90   
7     8                           Morgan Stanley          140.83   
8     9                  China Construction Bank          139.82   
9    10                            Bank of China          136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215.31  

In [63]:
!rm code_log.txt


In [64]:
# Display the contents of the log file
!cat code_log.txt


cat: code_log.txt: No such file or directory
